In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import pickle as pickle # for saving loss objects

import dataset as dd # custom dataset class
import models as md

# so that when you change an imported file, it changes in the notebook
%load_ext autoreload 
%autoreload 2
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Notebook cell for running all models we are interested in for spotlight talk trained over many epochs
epochs_to_train = 300

model_params_1 = [{'tag': 'pooling_channel_aug_small', 'use_pool': True, 'do_channel_augmentation': True, 
                 'model_fn': md.get_unet},
                {'tag': 'no_pooling_channel_aug_small', 'use_pool': False, 'do_channel_augmentation': True, 
                 'model_fn': md.get_unet}]

model_params_2 = [{'tag': 'pooling_no_channel_aug_small', 'use_pool': True, 'do_channel_augmentation': False, 
                 'model_fn': md.get_unet},
                {'tag': 'no_pooling_no_channel_aug_small', 'use_pool': False, 'do_channel_augmentation': False, 
                 'model_fn': md.get_unet},
                {'tag': 'kaist', 'do_channel_augmentation': False, 'model_fn': md.get_kaist_unet}
                ]

model_params = model_params_1 + model_params_2

for model_param in model_params:
    
    # make generators
    print(model_param['tag'])
    training_scans = [1, 2, 3, 4]
    testing_scans = [5]
    
    generator_train = dd.MRImageSequence(scan_numbers=training_scans, batch_size=10, 
                                         augment_channels=model_param['do_channel_augmentation'],
                                         augment_images=True)
    generator_test = dd.MRImageSequence(scan_numbers=testing_scans, batch_size=10, augment_channels=model_param['do_channel_augmentation'])    
        
    # make model
    input_shape = generator_test.x_transformed[0].shape[1:]
    inputs = tf.keras.layers.Input(shape=input_shape)

    if model_param['model_fn'] == md.get_unet:
        out = model_param['model_fn'](inputs, [(2, 32), (3, 64)], use_pool=model_param['use_pool'])        
    elif model_param['model_fn'] == md.get_kaist_unet:
        out = model_param['model_fn'](inputs)
    else:
        error()
        
    model = tf.keras.models.Model(inputs=inputs, outputs=out)
    
    # make callback
    history_callback = dd.LossHistory(test_data=(generator_test.x_transformed[0], generator_test.y_transformed[0]))
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir='/home/pkllee/tmp/')

    adam_optimizer = tf.keras.optimizers.Adam(lr=0.001, decay=0.01)
    
    model.compile(optimizer=adam_optimizer, loss='mean_squared_error', metrics=['mse'])
    
    # train model
    model.fit_generator(generator_train, callbacks=[history_callback, tb_callback], epochs=epochs_to_train, 
                    use_multiprocessing=False) # use_multiprocessing=True is slower by about 50% compared to model.fit() so set it to False
    
    
    # save model
    save_path_model = 'models/' + model_param['tag'] + '.h5'
    print(save_path_model)
    model.save(save_path_model)
    
    # save loss object
    loss_dict = {'train_losses_batch': history_callback.train_losses_batch,
                 'train_losses_epoch': history_callback.train_losses_epoch,
                 'test_losses': history_callback.test_losses}
    
    save_path_loss_object = 'models/' + model_param['tag'] + '_loss' + '.pkl'
    with open(save_path_loss_object, 'wb') as output:
        pickle.dump(loss_dict, output, pickle.HIGHEST_PROTOCOL)


pooling_channel_aug_small
('loading scan ', 1)
('X shape: ', (320, 320, 256, 16))
('y shape: ', (320, 320, 256, 1))
('augment_images: ', True)
('loading scan ', 5)
('X shape: ', (320, 320, 256, 16))
('y shape: ', (320, 320, 256, 1))
('augment_images: ', False)
get_unet
('use_pool: ', True)
('gen_fn: ', 'gen_conv_relu')
('unet_shape: ', [(2, 32), (3, 64)])
Epoch 1/10
32/32 [==============================] - 37s 1s/step - loss: 3.6098e-04 - mean_squared_error: 3.6098e-04
Epoch 2/10
32/32 [==============================] - 20s 617ms/step - loss: 1.3181e-04 - mean_squared_error: 1.3181e-04
Epoch 3/10
32/32 [==============================] - 18s 552ms/step - loss: 9.5692e-05 - mean_squared_error: 9.5692e-05
Epoch 4/10
32/32 [==============================] - 19s 597ms/step - loss: 7.4968e-05 - mean_squared_error: 7.4968e-05
Epoch 5/10
32/32 [==============================] - 17s 530ms/step - loss: 6.1362e-05 - mean_squared_error: 6.1362e-05
Epoch 6/10
32/32 [==============================] 

KeyboardInterrupt: 

In [ ]:
# cell for going backwards (loading data)
from keras.models import load_model

# load results
results = []

for model_param in model_params:
    print(model_param)
    save_path_model = 'models/' + model_param['tag'] + '.h5'    
    save_path_loss_object = 'models/' + model_param['tag'] + '_loss' + '.pkl'
    
    model = load_model(save_path_model)
    loss_dict = pickle.load(open(save_path_loss_object, 'rb'))
    
    results.append((model_param, model, loss_dict))    